In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Modeling Congestion and Pricing Transmission Rights in Coupled Power Markets

## Preprocessing des données

Pour le prétraitement des données, nous avons téléchargé les prix day-ahead de la France et de l’Allemagne sur ENTSO-E, puis fusionné les séries temporelles sur les dates communes avant de créer une nouvelle colonne représentant le spread horaire (différence entre le prix français et le prix allemand).

In [2]:
df = pd.read_csv('clean_data.csv')

## Identification des patterns de congestion

In [8]:
threshold_value = df['spread'].abs().quantile(0.95)
print(f"Seuil de congestion : {threshold_value} €/MWh")

# On essaye de déterminer les événements de congestion
df['congestion'] = df['spread'].abs() > threshold_value
df['congestion_sign'] = np.sign(df['spread']) * df['congestion']

# On détecte les groupes consécutifs
df['event_id'] = (df['congestion'] != df['congestion'].shift(1)).cumsum()
congestion_events = df[df['congestion']].groupby('event_id')

Seuil de congestion : 79.16550000000001 €/MWh


### Statistiques

In [9]:
event_stats = []
for event_id, group in congestion_events:
    event_stats.append({
        'event_id': event_id,
        'start_time': group['MTU (UTC)'].iloc[0],
        'end_time': group['datetime'].iloc[-1],
        'duration_hours': len(group),
        'mean_spread': group['spread'].mean(),
        'max_spread': group['spread'].abs().max(),
        'min_spread': group['spread'].abs().min(),
        'direction': 'FR > DE' if group['spread'].mean() > 0 else 'DE > FR',
        'magnitude_avg': group['spread'].abs().mean(),
        'magnitude_max': group['spread'].abs().max()
    })

events_df = pd.DataFrame(event_stats)

# Résumé
print("\n Statistiques de congestion")
print(f"Nombre total d'événements : {len(events_df)}")
print(f"Fréquence : {len(events_df) / (df['datetime'].max() - df['datetime'].min()).days * 30:.2f} événements par mois")
print(f"Durée moyenne : {events_df['duration_hours'].mean():.2f} heures")
print(f"Durée médiane : {events_df['duration_hours'].median():.2f} heures")
print(f"Magnitude moyenne (abs) : {events_df['magnitude_avg'].mean():.2f} €/MWh")
print(f"Magnitude max (abs) : {events_df['magnitude_max'].max():.2f} €/MWh")

# Répartition par direction
direction_counts = events_df['direction'].value_counts()
print("\nDirection des congestions :")
print(direction_counts)

KeyError: 'datetime'

## Visualisation

Pour avoir une meilleure idée de ce qui se passe réellement, on visualise nos résultats sous la forme de graphique

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

### Histogramme des durées

In [ ]:
axes[0,0].hist(events_df['duration_hours'], bins=30, edgecolor='black', alpha=0.7)
axes[0,0].axvline(events_df['duration_hours'].mean(), color='red', linestyle='--', label=f'Moyenne: {events_df["duration_hours"].mean():.1f}h')
axes[0,0].set_xlabel('Durée (heures)')
axes[0,0].set_ylabel('Fréquence')
axes[0,0].set_title('Distribution des durées de congestion')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

### Histogramme des magnitudes moyennes

In [ ]:
axes[0,1].hist(events_df['magnitude_avg'], bins=30, edgecolor='black', alpha=0.7, color='orange')
axes[0,1].axvline(events_df['magnitude_avg'].mean(), color='red', linestyle='--', label=f'Moyenne: {events_df["magnitude_avg"].mean():.1f}€')
axes[0,1].set_xlabel('Spread moyen (€/MWh)')
axes[0,1].set_ylabel('Fréquence')
axes[0,1].set_title('Distribution des magnitudes moyennes')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

### Diagramme en barre des directions

In [ ]:
direction_counts.plot(kind='bar', ax=axes[0,2], color=['blue', 'red'])
axes[0,2].set_xlabel('Direction')
axes[0,2].set_ylabel('Nombre d\'événements')
axes[0,2].set_title('Direction des congestions')
axes[0,2].tick_params(axis='x', rotation=0)

### Durée vs Magnitude

In [ ]:
scatter = axes[1,0].scatter(events_df['duration_hours'], events_df['magnitude_avg'],
                           c=events_df['magnitude_max'], cmap='viridis', alpha=0.6)
axes[1,0].set_xlabel('Durée (heures)')
axes[1,0].set_ylabel('Spread moyen (€/MWh)')
axes[1,0].set_title('Durée vs Magnitude moyenne')
plt.colorbar(scatter, ax=axes[1,0], label='Spread max (€/MWh)')
axes[1,0].grid(True, alpha=0.3)

### Saisonnalité des événements (par mois)

In [ ]:
events_df['month'] = events_df['start_time'].dt.month
monthly_counts = events_df.groupby('month').size()
axes[1,1].plot(monthly_counts.index, monthly_counts.values, marker='o', linewidth=2)
axes[1,1].set_xlabel('Mois')
axes[1,1].set_ylabel('Nombre d\'événements')
axes[1,1].set_title('Saisonnalité des congestions (par mois)')
axes[1,1].set_xticks(range(1, 13))
axes[1,1].grid(True, alpha=0.3)

### Saisonnalité horaire

In [ ]:
events_df['hour'] = events_df['start_time'].dt.hour
hourly_counts = events_df.groupby('hour').size()
axes[1,2].bar(hourly_counts.index, hourly_counts.values, alpha=0.7)
axes[1,2].set_xlabel('Heure de début')
axes[1,2].set_ylabel('Nombre d\'événements')
axes[1,2].set_title('Distribution horaire des débuts de congestion')
axes[1,2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
print("\n=== 10 ÉVÉNEMENTS DE CONGESTION LES PLUS INTENSES ===")
top_events = events_df.nlargest(10, 'magnitude_max')[['start_time', 'end_time', 'duration_hours',
                                                       'mean_spread', 'max_spread', 'direction']]
print(top_events.to_string())

# --- 9. Export des résultats ---
events_df.to_csv('congestion_events_analysis.csv', index=False)
print("\nRésultats exportés dans 'congestion_events_analysis.csv'")


### Spread pendant et hors congestion

In [ ]:
# --- 11. Distribution des spreads pendant congestion vs normal ---
fig2, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Spreads pendant congestion
congestion_spreads = df[df['congestion']]['spread']
ax1.hist(congestion_spreads, bins=50, alpha=0.7, edgecolor='black', color='red')
ax1.set_xlabel('Spread (€/MWh)')
ax1.set_ylabel('Fréquence')
ax1.set_title('Distribution des spreads PENDANT congestion')
ax1.grid(True, alpha=0.3)

# Spreads hors congestion
normal_spreads = df[~df['congestion']]['spread']
ax2.hist(normal_spreads, bins=50, alpha=0.7, edgecolor='black', color='green')
ax2.set_xlabel('Spread (€/MWh)')
ax2.set_ylabel('Fréquence')
ax2.set_title('Distribution des spreads HORS congestion')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#Correlation
correlation = events_df['duration_hours'].corr(events_df['magnitude_avg'])
print(f"\nCorrélation durée-magnitude : {correlation:.3f}")